In [1]:
using RigidBodyDynamics
using ForwardDiff
using Plots
using MeshCatMechanisms

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [157]:
# Double Pendulum
urdf = "cartpole.urdf"
cartpole = parse_urdf(Float64,urdf)
state = MechanismState(cartpole)

n = 4 # number of states
m = 1 # number of controls

# initial and goal states
x0 = [.1;0;0.;0.]
xf = [.5;0.;0.;0.]

#set_configuration!(state, x0[1:2])
#set_velocity!(state, x0[3:4])

# costs
Q = 5e-4*eye(n)
Qf = 500.0*eye(n)
R = 1e-5*eye(m)

# simulation
tf = 1.0
dt = 0.01

0.01

In [158]:
fc(xf,1)
f(xf,1,dt)
f_aug([xf;1;dt])

    

6-element Array{Float64,1}:
  0.5        
 -0.000990344
  9.77697e-5 
 -0.198069   
  1.0        
  0.01       

In [164]:
# Dynamics (RK4 discretization)
function fc(x,u)
    g = 9.81
    m1 = .35
    m2 = 2
    l = 0.5

    z = x[1];
    theta = x[2];
    zdot = x[3];
    thetadot = x[4];
    
    F_tilde = m1/2 + m2*z/l
    F = u[1] + F_tilde*0;
    
    zddot = z*thetadot^2-g*sin(theta);
    thetaddot = (F*l*cos(theta) - 2*m1*z*zdot*thetadot - m1*g*z*cos(theta) - (m2*g*l*cos(theta))/2) / (m2*l^2/3 + m1*z^2);

    sys = [zdot; thetadot; zddot; thetaddot];
end

function f(x,u,dt)
    # Runge-Kutta 4
    k1 = dt*fc(x,u)
    k2 = dt*fc(x + k1/2,u)
    k3 = dt*fc(x + k2/2,u)
    k4 = dt*fc(x + k3, u)
    return x + (k1 + 2*k2 + 2*k3 + k4)/6
end

function fc_aug(S)
    g = 9.81
    m1 = .35
    m2 = 2
    l = 0.5

    z = S[1];
    theta = S[2];
    zdot = S[3];
    thetadot = S[4];
    
    F_tilde = m1/2 + m2*z/l
    F = S[5] + F_tilde*0;
    
    zddot = z*thetadot^2-g*sin(theta);
    thetaddot = (F*l*cos(theta) - 2*m1*z*zdot*thetadot - m1*g*z*cos(theta) - (m2*g*l*cos(theta))/2) / (m2*l^2/3 + m1*z^2);

    sys = [zdot; thetadot; zddot; thetaddot;0;0];
end

function f_aug(S)
    k1 = S[end]*fc_aug(S)
    k2 = S[end]*fc_aug(S + k1/2)
    k3 = S[end]*fc_aug(S + k2/2)
    k4 = S[end]*fc_aug(S + k3)
    return S + (k1 + 2*k2 + 2*k3 + k4)/6
end

F = S-> ForwardDiff.jacobian(f_aug, S)

(::#81) (generic function with 1 method)

In [165]:
#iLQR
function rollout(x0,U,f,dt,tf)
    N = convert(Int64,floor(tf/dt))
    X = zeros(size(x0,1),N)
    X[:,1] = x0
    for k = 1:N-1
      X[:,k+1] = f(X[:,k],U[:,k],dt)
    end
    return X
end

function cost(X,U,Q,R,Qf,xf)
    N = size(X,2)
    J = 0.0
    for k = 1:N-1
      J += 0.5*(X[:,k] - xf)'*Q*(X[:,k] - xf) + 0.5*U[:,k]'*R*U[:,k]
    end
    J += 0.5*(X[:,N] - xf)'*Qf*(X[:,N] - xf)
    return J
end

function backwardpass(X,U,F,Q,R,Qf,xf)
    n, N = size(X)
    m = size(U,1)
    
    S = zeros(n,n,N)
    s = zeros(n,N)
    
    Qx = zeros(n,1,N)
    Qu = zeros(m,1,N)
    Qxx = zeros(n,n,N)
    Quu = zeros(m,m,N)
    Qux = zeros(m,n,N)
    
    K = zeros(m,n,N-1)
    l = zeros(m,N-1)
    
    S[:,:,N] = Qf
    s[:,N] = Qf*(X[:,N] - xf)
    v1 = 0.0
    v2 = 0.0

    mu = 0.0
    k = N-1
    
    while k >= 1
        q = Q*(X[:,k] - xf)
        r = R*(U[:,k])
        F_aug = F([X[:,k];U[:,k];dt])
        fx = F_aug[1:size(X,1),1:size(X,1)]
        fu = F_aug[1:size(X,1),size(X,1)+1:size(X,1)+size(U,1)]
        
        Qx[:,:,k] = q + fx'*s[:,k+1]
        Qu[:,:,k] = r + fu'*s[:,k+1]
        Qxx[:,:,k] = Q + fx'*S[:,:,k+1]*fx
        Quu[:,:,k] = R + fu'*(S[:,:,k+1] + mu*eye(n))*fu
        Qux[:,:,k] = fu'*(S[:,:,k+1] + mu*eye(n))*fx

        # regularization        
        if any(x->x < 0.0, (eigvals(Quu[:,:,k])))
            mu = mu + 1.0;
            k = N-1;
            println("regularized")
        end
        
        K[:,:,k] = Quu[:,:,k]\Qux[:,:,k]
        l[:,k] = Quu[:,:,k]\Qu[:,:,k]
        s[:,k] = (Qx[:,:,k]' - Qu[:,:,k]'*K[:,:,k] + l[:,k]'*Quu[:,:,k]*K[:,:,k] - l[:,k]'*Qux[:,:,k])'
        S[:,:,k] = Qxx[:,:,k] + K[:,:,k]'*Quu[:,:,k]*K[:,:,k] - K[:,:,k]'*Qux[:,:,k] - Qux[:,:,k]'*K[:,:,k]

        # terms for line search
        v1 += l[:,k]'*Qu[:,:,k]
        v2 += l[:,k]'*Quu[:,:,k]*l[:,k]
        
        k = k - 1;
    end
    return K, l, v1, v2
end

function forwardpass(X,U,f,J,K,l,v1,v2,c1=0.5,c2=.85)
    N = size(X,2)
    m = size(U,1)
    X_prev = copy(X)
    J_prev = copy(J)
    U_ = zeros(m,N-1)
    J = Inf
    dV = 0.0
    dJ = 0.0
    z = 0.0
    
    alpha = 1.0

    while J > J_prev || z < c1 || z > c2 
        for k = 1:N-1
          U_[:,k] = U[:,k] - K[:,:,k]*(X[:,k] - X_prev[:,k]) - alpha*l[:,k]
          X[:,k+1] = f(X[:,k],U_[:,k],dt);
        end

        J = cost(X,U_,Q,R,Qf,xf)
        
        dV = alpha*v1 + (alpha^2)*v2/2.0
        dJ = J_prev - J
        z = dJ/dV[1]

        @show alpha = alpha/2.0;
    end

    println("New cost: $J")
    println("- Expected improvement: $(dV[1])")
    println("- Actual improvement: $(dJ)")
    println("- (z = $z)\n")
    
    return X, U_, J
end

function solve(x0,m,f,F,Q,R,Qf,xf,dt,tf,iterations=50,eps=1e-3; control_init="random")
    N = convert(Int64,floor(tf/dt))
    X = zeros(size(x0,1),N)
    
    if control_init == "random"
        U = 10.0*ones(m,N-1)
    else
        U = zeros(m,N-1)
    end
        
    X = rollout(x0,U,f,dt,tf)
    J_prev = cost(X,U,Q,R,Qf,xf)
    println("Initial Cost: $J_prev\n")
    
    for i = 1:iterations
        println("*** Iteration: $i ***")
        K, l, v1, v2 = backwardpass(X,U,F,Q,R,Qf,xf)
        X, U, J = forwardpass(X,U,f,J_prev,K,l,v1,v2)
        
        if abs(J-J_prev) < eps
          println("-----SOLVED-----")
          println("eps criteria met at iteration: $i")
          break
        end
        J_prev = copy(J)
    end
    
    return X, U
end

solve (generic function with 3 methods)

In [167]:
X, U = @time solve(x0,m,f,F,Q,R,Qf,xf,dt,tf;control_init="random");

Initial Cost: 2823.2494176311397

*** Iteration: 1 ***
alpha = alpha / 2.0 = 0.5
alpha = alpha / 2.0 = 0.25
New cost: 599.990675328382
- Expected improvement: 3528.6544594077964
- Actual improvement: 2223.2587423027576
- (z = 0.6300585018675591)

*** Iteration: 2 ***
alpha = alpha / 2.0 = 0.5
alpha = alpha / 2.0 = 0.25
New cost: 158.42253147955947
- Expected improvement: 749.8347262173195
- Actual improvement: 441.5681438488226
- (z = 0.5888872953062538)

*** Iteration: 3 ***
alpha = alpha / 2.0 = 0.5
alpha = alpha / 2.0 = 0.25
New cost: 56.573229784142185
- Expected improvement: 197.90499921212552
- Actual improvement: 101.84930169541728
- (z = 0.5146373366053758)

*** Iteration: 4 ***
alpha = alpha / 2.0 = 0.5
alpha = alpha / 2.0 = 0.25
New cost: 15.049480537496956
- Expected improvement: 70.53794243595465
- Actual improvement: 41.52374924664523
- (z = 0.5886725330037372)

*** Iteration: 5 ***
alpha = alpha / 2.0 = 0.5
alpha = alpha / 2.0 = 0.25
New cost: 3.891600571350444
- Expected

In [154]:
X
Int(round(length(anim.frames)/tf))

34

In [168]:
plt = plot()
N = size(X,2)
l = .6
anim = @animate for i = 1:N-1
    z = X[1,i]
    theta = X[2,i]
    plt = plot([0, l*cos(theta)],[0, l*sin(theta)])
    plot!([z*cos(theta)], [z*sin(theta)], seriestype=:scatter, markersize=2,
        xlims=(-l/5,1.5l),ylims=(-l/5,1.5l),size=(200,200),label="",title="Ball on Beam")
end every 3

Plots.Animation("C:\\Users\\bjack\\AppData\\Local\\Temp\\jl_112.tmp", String["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000024.png", "000025.png", "000026.png", "000027.png", "000028.png", "000029.png", "000030.png", "000031.png", "000032.png", "000033.png"])

In [169]:
gif(anim, "BallOnBeam.gif", fps=Int(round(length(anim.frames)/tf)))

INFO: Saved animation to C:\Users\bjack\Documents\GitHub\JuliaiLQR\iLQR demos\BallOnBeam.gif


Plots.AnimatedGif("C:\\Users\\bjack\\Documents\\GitHub\\JuliaiLQR\\iLQR demos\\BallOnBeam.gif")

In [ ]:
P = plot(linspace(0,tf,size(X,2)),X[1,:],title="Cartpole",label="y")
P = plot!(linspace(0,tf,size(X,2)),X[2,:],ylabel="State",label="\Theta")

In [ ]:
W = plot(linspace(0,tf,size(X,2)),[U[1,1] U[1,:]']',title="Cartpole")
W = plot!(linspace(0,tf,size(X,2)),[U[2,1] U[2,:]']',ylabel="Control")

In [ ]:
vis = MechanismVisualizer(cartpole,URDFVisuals(urdf));
IJuliaCell(vis)

In [ ]:
for i = 1:size(X,2)
    set_configuration!(state, X[1:2,i])
    set_configuration!(vis, configuration(state))
    sleep(dt)
end